# Prepping the Data
We need to
- [x] remove high null columns and rows
- [x] remove unnecessary columns
- [x] remove outliers
- [x] impute/remove leftover nulls
- [x] create new features
- [X] reorder/rename columns
- [ ] put into one function

Preprocessing
- [ ] scale the data
- [ ] split into train, validate, test (stratify)

In [1]:
import acquire

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = acquire.get_home_data()

In [3]:
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,0.055619,2017-01-01
1,6,10933547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,404013.0,563029.0,2016.0,159016.0,6773.34,NaN,NaN,NaN,-0.001011,2017-01-01
2,14,11142747,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,4265.0,2015.0,4265.0,NaN,NaN,NaN,NaN,-0.008935,2017-01-02
3,15,11193347,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,10.0,2016.0,10.0,NaN,NaN,NaN,NaN,0.008669,2017-01-02
4,16,11215747,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,10.0,2016.0,10.0,NaN,NaN,NaN,NaN,-0.021896,2017-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70359,77609,11212539,1.0,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,129566.0,162019.0,2016.0,32453.0,2860.33,NaN,NaN,6.037911e+13,0.020615,2017-09-20
70360,77610,11212639,1.0,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,100744.0,125923.0,2016.0,25179.0,2394.26,NaN,NaN,6.037911e+13,0.013209,2017-09-21
70361,77611,11212962,1.0,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,149241.0,198988.0,2016.0,49747.0,3331.81,NaN,NaN,6.037911e+13,0.037129,2017-09-21
70362,77612,11213162,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,118900.0,148600.0,2016.0,29700.0,2510.53,NaN,NaN,6.037911e+13,0.007204,2017-09-25


# Remove Nulls in Columns/Rows

In [4]:
# sets thresh hold to 75 percent nulls
threshold = df.shape[0] * .75

# remove columns with high nulls
df = df.dropna(axis=1, thresh=threshold)

df.columns

Index(['id', 'parcelid', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'logerror', 'transactiondate'],
      dtype='object')

In [6]:
# sets thresh hold to 25 percent nulls
thresh_hold = df.shape[1] * .75

# remove rows with high nulls
df = df.dropna(axis=0,thresh=thresh_hold)

### Removing repeated/unecessary columns
- multiple sqft columns
- multiple county ids
- multiple census tract and block ids

In [5]:
# remove additional sqft
df = df.drop(columns=['finishedsquarefeet12'])

In [114]:
# removing one of the county id columns
df.regionidcounty.value_counts()

3101.0    43921
1286.0    14041
2061.0     5122
Name: regionidcounty, dtype: int64

In [115]:
df.fips.value_counts()

6037.0    43921
6059.0    14041
6111.0     5122
Name: fips, dtype: int64

In [117]:
# region county id and fips are the same, removing longer columns name
df = df.drop('regionidcounty',axis=1)

In [123]:
df.rawcensustractandblock.value_counts()

6.037920e+07    28
6.037274e+07    25
6.037142e+07    24
6.037920e+07    23
6.037277e+07    23
                ..
6.037555e+07     1
6.037431e+07     1
6.037402e+07     1
6.037651e+07     1
6.037920e+07     1
Name: rawcensustractandblock, Length: 40910, dtype: int64

In [124]:
df.censustractandblock.value_counts()

6.037142e+13    261
6.037920e+13     23
6.037920e+13     23
6.037920e+13     23
6.037277e+13     23
               ... 
6.037409e+13      1
6.037430e+13      1
6.037620e+13      1
6.111004e+13      1
6.037540e+13      1
Name: censustractandblock, Length: 40846, dtype: int64

In [125]:
# region county id and fips are the same, removing longer columns name
df = df.drop('rawcensustractandblock',axis=1)

# Removing Outliers Using IQR Rule
- Before outliers have 70,364 rows
- After, we have 63,084

In [11]:
columns = ['calculatedfinishedsquarefeet','lotsizesquarefeet','structuretaxvaluedollarcnt',
           'landtaxvaluedollarcnt','taxamount']

for x in columns:
    
    Q1 = df[x].quantile(0.25)
    Q3 = df[x].quantile(0.75)
    IQR = (Q3 - Q1) * 1.5
    upper = Q3 + (1.5 * IQR)
    lower = Q1 - (1.5 * IQR)
    
    print('column:', x,'\nIQR:', IQR, '\nUpper bound:', upper, '\nLower bound:', lower, '\n')

column: calculatedfinishedsquarefeet 
IQR: 1324.5 
Upper bound: 4068.75 
Lower bound: -787.75 

column: lotsizesquarefeet 
IQR: 6632.25 
Upper bound: 20205.875 
Lower bound: -4112.375 

column: structuretaxvaluedollarcnt 
IQR: 188069.25 
Upper bound: 485437.125 
Lower bound: -204150.125 

column: landtaxvaluedollarcnt 
IQR: 374597.625 
Upper bound: 889017.4375 
Lower bound: -484507.1875 

column: taxamount 
IQR: 5561.752500000001 
Upper bound: 14518.148750000002 
Lower bound: -5874.943750000002 



In [28]:
for x in columns:
    
    Q1 = df[x].quantile(0.25)
    Q3 = df[x].quantile(0.75)
    IQR = (Q3 - Q1) * 1.5
    upper = Q3 + (1.5 * IQR)
    lower = Q1 - (1.5 * IQR)
    
    df = df[(df[x] > (lower)) | (df[x] < (upper))]

In [30]:
df.shape

(63084, 27)

# Exploring leftover nulls to determine if dropping or imputing with mean, median, mode

In [31]:
df.isnull().sum()

id                                 0
parcelid                           0
bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr                166
calculatedfinishedsquarefeet       0
fips                               0
fullbathcnt                      166
latitude                           0
longitude                          0
lotsizesquarefeet                  0
propertycountylandusecode          0
propertylandusetypeid              0
rawcensustractandblock             0
regionidcity                    1055
regionidcounty                     0
regionidzip                       41
roomcnt                            0
yearbuilt                         35
structuretaxvaluedollarcnt         0
taxvaluedollarcnt                  0
assessmentyear                     0
landtaxvaluedollarcnt              0
taxamount                          0
censustractandblock              237
logerror                           0
transactiondate                    0
d

### Calculatedbathnbdr will be dealt with when creating features later
### Fullbathcnt

In [32]:
df.fullbathcnt.value_counts()

2.0     31523
3.0     14541
1.0     12661
4.0      2738
5.0       931
6.0       322
7.0       123
8.0        39
9.0        24
10.0        9
11.0        3
20.0        1
13.0        1
19.0        1
12.0        1
Name: fullbathcnt, dtype: int64

- Fullbathcnt has 166 nulls
- Most common value 2 has 35,694 observations
- Will impute the nulls with this

In [33]:
mode = df.fullbathcnt.mode()[0]

df['fullbathcnt'] = df.fullbathcnt.fillna(mode)

In [36]:
df['fullbathcnt'] = df.fullbathcnt.fillna(2)

### Region ID City
- has 1,055 nulls
- we already have latitude and longitude with no nulls for location
- will drop this column

In [37]:
df = df.drop('regionidcity',axis=1)

### Region ID Zip
- will replace nulls with 90000 to represent no known zip code (but not create outliers by using 0)
- can use latitude/longitude or clustering if necessary to determine actual values
- however, for only 41 nulls it is not a significant amount to worry about

In [38]:
df.regionidzip.value_counts()

96193.0    519
96368.0    517
97118.0    512
97319.0    479
96361.0    473
          ... 
95991.0      4
96226.0      2
96467.0      1
97177.0      1
96963.0      1
Name: regionidzip, Length: 385, dtype: int64

In [39]:
df['regionidzip'] = df.regionidzip.fillna(90_000)

### Year Built
- 35 nulls
- not many nulls to be significant, will replace with mean

In [40]:
df.yearbuilt.value_counts(), df.yearbuilt.mean()

(1955.0    2174
 1950.0    1850
 1954.0    1843
 1956.0    1704
 1953.0    1596
           ... 
 1862.0       1
 1880.0       1
 1889.0       1
 2016.0       1
 1893.0       1
 Name: yearbuilt, Length: 133, dtype: int64,
 1963.5353931069485)

In [43]:
df['yearbuilt'] = df.yearbuilt.fillna(1964)

### censustractandblock
- 237 nulls is not significant
- will replace with mode

In [44]:
df.censustractandblock.value_counts(), df.censustractandblock.mode()

(6.037142e+13    24
 6.037920e+13    23
 6.037920e+13    23
 6.037920e+13    23
 6.037277e+13    23
                 ..
 6.037409e+13     1
 6.037430e+13     1
 6.037620e+13     1
 6.111004e+13     1
 6.037540e+13     1
 Name: censustractandblock, Length: 40846, dtype: int64,
 0    6.037142e+13
 dtype: float64)

In [46]:
mode = df.censustractandblock.mode()[0]

df['censustractandblock'] = df.censustractandblock.fillna(mode)

# Create Features

### calculate our own bath_bed
- we have a column from the database, however it has 215 nulls
- bathroom and bedroom count by themselves have no nulls
- calculate our own and drop the original

In [48]:
df['bed_plus_bath'] = df.bathroomcnt + df.bedroomcnt
df = df.drop('calculatedbathnbr',axis=1)

In [49]:
df.isnull().sum()

id                              0
parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
propertylandusetypeid           0
rawcensustractandblock          0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
logerror                        0
transactiondate                 0
bed_plus_bath                   0
dtype: int64

### All our nulls and outliers are removed, what other features could we create?

## How many years has the house been around?
- calculate by todays year - year built

In [51]:
df['age'] = 2020 - df.yearbuilt

## Dummy variables for year assessed?
- could say assesed in 2016 or not
- however, only 20 properties were not assessed in 2016
- we can drop this feature

In [53]:
df.assessmentyear.value_counts()

2016.0    63064
2014.0       18
2015.0        2
Name: assessmentyear, dtype: int64

In [55]:
df = df.drop('assessmentyear',axis=1)

## Transaction date by months? Range of 1-12

In [84]:
df.transactiondate.value_counts()

2017-06-30    1061
2017-05-31     845
2017-04-28     768
2017-07-28     763
2017-05-26     712
              ... 
2017-03-12       1
2017-02-18       1
2018-05-25       1
2017-05-20       1
2017-08-06       1
Name: transactiondate, Length: 256, dtype: int64

In [109]:
# converting to string to use split method
df['transactiondate'] = df.transactiondate.astype('str')

# creating new feature as the second index (month) of the transacion date split
df['transaction_month'] = df.transactiondate.str.split('-',expand=True)[1]

# Reorder/Rename Columns for Better Intuition

In [126]:
df.columns

Index(['id', 'parcelid', 'bathroomcnt', 'bedroomcnt',
       'calculatedfinishedsquarefeet', 'fips', 'fullbathcnt', 'latitude',
       'longitude', 'lotsizesquarefeet', 'propertycountylandusecode',
       'propertylandusetypeid', 'regionidzip', 'roomcnt', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock', 'logerror',
       'transactiondate', 'bed_plus_bath', 'age', 'transaction_month'],
      dtype='object')

In [127]:
df.columns = ['index_id','parcel_id','bathrooms','bedrooms','property_sqft','county_id','full_bathrooms','latitude',
             'longitude','lot_sqft','land_use_code','land_use_type','zip_code','room_count','year_built',
             'structure_tax_value','tax_value','land_tax_value','tax_amount', 'census_id','log_error',
              'transaction_date','bed_plus_bath','property_age','transaction_month'
             ]

In [128]:
df.columns

Index(['index_id', 'parcel_id', 'bathrooms', 'bedrooms', 'property_sqft',
       'county_id', 'full_bathrooms', 'latitude', 'longitude', 'lot_sqft',
       'land_use_code', 'land_use_type', 'zip_code', 'room_count',
       'year_built', 'structure_tax_value', 'tax_value', 'land_tax_value',
       'tax_amount', 'census_id', 'log_error', 'transaction_date',
       'bed_plus_bath', 'property_age', 'transaction_month'],
      dtype='object')

In [130]:
# Reordering columns
df = df[['index_id', 'parcel_id',
    'log_error', 'tax_value', 'structure_tax_value', 'land_tax_value', 'tax_amount',
    'county_id', 'zip_code', 'latitude', 'longitude', 'census_id',
    'bathrooms', 'bedrooms', 'full_bathrooms', 'bed_plus_bath', 'room_count',
    'property_sqft', 'lot_sqft',
    'land_use_code', 'land_use_type',
    'year_built', 'property_age', 'transaction_date', 'transaction_month'
   ]]

# Put all together into a function for the Prepare.py module

In [133]:
def prepare_zillow():
    '''
    Acquire and prepare the zillow data obtained from the SQL database.
    Nulls are removed/replaced, outliers are removed, new features are created,
    and columns are renamed/rearranged. Returns the prepped df.
    '''
    # acquire the data from module
    df = acquire.get_home_data()
    
    # Removing Nulls from Columns
    # sets thresh hold to 75 percent nulls, if more than %25 nulls it will be removed
    threshold = df.shape[0] * .75

    # remove columns with specified threshold
    df = df.dropna(axis=1, thresh=threshold)
    
    # Removing Nulls from Rows
    # sets thresh hold to 75 percent nulls, if more than %25 nulls it will be removed
    thresh_hold = df.shape[1] * .75

    # remove rows with specified threshold
    df = df.dropna(axis=0,thresh=thresh_hold)
    
    # Removing Columns with Repeated Data/Unecessary Data
    # don't need additional sqft, county id/city, assessment year, and census columns
    df = df.drop(columns=['finishedsquarefeet12', 'regionidcounty', 'rawcensustractandblock',
                          'regionidcity','assessmentyear'], axis=1)
    
    # Removing Outliers from Continuous Variables
    # assigning columns to remove outliers
    columns = ['calculatedfinishedsquarefeet','lotsizesquarefeet','structuretaxvaluedollarcnt',
           'landtaxvaluedollarcnt','taxamount']
    
    # looping through continuous variables to remove outliers
    for x in columns:
    
        # calculate IQR
        Q1 = df[x].quantile(0.25)
        Q3 = df[x].quantile(0.75)
        IQR = (Q3 - Q1) * 1.5
        
        # calculate upper and lower bounds, outlier if above or below these
        upper = Q3 + (1.5 * IQR)
        lower = Q1 - (1.5 * IQR)
    
        # creates df of values that are within the outlier bounds
        df = df[(df[x] > (lower)) | (df[x] < (upper))]
        
    # Filling Leftover Nulls by Columns
    # Full Bathroom Count Nulls
    # mode of bathroomcnt
    fullbath_mode = df.fullbathcnt.mode()[0]
    # filling nulls with the mode
    df['fullbathcnt'] = df.fullbathcnt.fillna(fullbath_mode)
    
    # Region Zip Code Nulls
    # filling with 90000 to represent no known zipcode (0 would skew the data)
    df['regionidzip'] = df.regionidzip.fillna(90_000)
    
    # Year Built Nulls
    # average of property year built
    year_avg = round(df.yearbuilt.mean())
    # filling nulls with average year built
    df['yearbuilt'] = df.yearbuilt.fillna(year_avg)
    
    # Census Tract and Block Nulls
    # mode of census tract and block
    census_mode = df.censustractandblock.mode()[0]
    # filling nulls with mode
    df['censustractandblock'] = df.censustractandblock.fillna(mode)
    
    # Feature Engineering - creating columns
    # calculating bed+bath from 0 null columns of bedroom/bathroom count
    df['bed_plus_bath'] = df.bathroomcnt + df.bedroomcnt
    # droping original calculated field that had nulls
    df = df.drop('calculatedbathnbr',axis=1)
    
    # Property Age
    # current year minus year built
    df['age'] = 2020 - df.yearbuilt
    
    # Transaction Month
    # converting date to string to use split method
    df['transactiondate'] = df.transactiondate.astype('str')
    # creating new feature as the second index (month) of the transaction date split
    df['transaction_month'] = df.transactiondate.str.split('-',expand=True)[1]
    
    # Renaming Columns
    df.columns = ['index_id', 'parcel_id', 'bathrooms', 'bedrooms', 'property_sqft', 'county_id', 'full_bathrooms',
                  'latitude', 'longitude', 'lot_sqft', 'land_use_code', 'land_use_type', 'zip_code', 'room_count',
                  'year_built', 'structure_tax_value', 'tax_value', 'land_tax_value', 'tax_amount', 'census_id',
                  'log_error', 'transaction_date', 'bed_plus_bath', 'property_age', 'transaction_month'
             ]
    
    # Reordering Columns
    df = df[['index_id', 'parcel_id',
        'log_error', 'tax_value', 'structure_tax_value', 'land_tax_value', 'tax_amount',
        'county_id', 'zip_code', 'latitude', 'longitude', 'census_id',
        'bathrooms', 'bedrooms', 'full_bathrooms', 'bed_plus_bath', 'room_count',
        'property_sqft', 'lot_sqft',
        'land_use_code', 'land_use_type',
        'year_built', 'property_age', 'transaction_date', 'transaction_month'
       ]]
    
    return df